# ASMSA: Train AAE model with the tuned hyperparameters

**Previous steps**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule

**Next step**
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

## Notebook setup

In [ ]:
threads = 2
import os
os.environ['OMP_NUM_THREADS']=str(threads)
import tensorflow as tf

# PyTorch favours OMP_NUM_THREADS in environment
import torch

# Tensorflow needs explicit cofig calls
tf.config.threading.set_inter_op_parallelism_threads(threads)
tf.config.threading.set_intra_op_parallelism_threads(threads)

In [ ]:
from asmsa.tuning_analyzer import TuningAnalyzer
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import urllib.request
from tensorflow import keras
import keras_tuner
import asmsa.visualizer as visualizer
import asmsa

In [ ]:
print(asmsa.__file__) 

## Input files

All input files are prepared (up- or downloaded) in [prepare.ipynb](prepare.ipynb). 


In [ ]:
exec(open('inputs.py').read())

## Apply the tuning results

In [ ]:
# pick from plots in tune.ipynb

best_enc_seed=128
best_disc_seed=32

In [ ]:
# Get best HP from latest tuning
analyzer = TuningAnalyzer()
analyzer.get_best_hp(num_trials=3)

In [ ]:
# Select HP to use by specifying trial_id
#  e.g: trial_id = '483883b929b3445bff6dee9759c4d50ee3a4ba7f0db22e665c49b5f942d9693b'
# ... or don't specify, by default use the trial with the lowest score
trial_id = 'abae5f92717a95f68c7f7560850cef4a5fd75cd6ff0b0639ce6bb8324bf15efa'

hps = None
for trial in analyzer.sorted_trials:
    if trial['trial_id'] == trial_id:
        hps = trial['hp']
    
if not hps:
    print(f'Could not find trial with specified ID, using one with the lowest score - {analyzer.sorted_trials[0]["trial_id"]}')
    hps = analyzer.sorted_trials[0]['hp']
    
print(hps)

## Load datasets
Load filtered trajectory datasets that were processed in **prepare.ipynb**. Trajectories are in internal coordinates format.

In [ ]:
# load train dataset
X_train = tf.data.Dataset.load('datasets/intcoords/train')

# get batched version of dataset to feed to AAE model for training
X_train_batched = X_train.batch(hps['batch_size'],drop_remainder=True)

# get numpy version for visualization purposes
X_train_np = np.stack(list(X_train))
X_train_np.shape

In [ ]:
# load test dataset
X_test = tf.data.Dataset.load('datasets/intcoords/test')

# get batched version of dataset to feed to AAE model for prediction
X_test_batched = X_test.batch(hps['batch_size'],drop_remainder=True)

# get numpy version for testing purposes
X_test_np = np.stack(list(X_test))
X_test_np.shape

## Train

### Distribution prior
Train with common prior distributions. See https://www.tensorflow.org/probability/api_docs/python/tfp/distributions for all available distributions. It is ideal to use tuned Hyperparameters for training.

In [ ]:
# set used prior

# this one is (more or less) required to with the density alignment
#prior = tfp.distributions.MultivariateNormalDiag(loc=[0.,0.])

prior = tfp.distributions.Normal(loc=0, scale=1)
# prior = tfp.distributions.Uniform()
# prior = tfp.distributions.Weibull(1,0.5)
# prior = tfp.distributions.Cauchy(loc=0, scale=1)

In [ ]:
# prepare model using the best hyperparameters
testm = asmsa.AAEModel((X_train_np.shape[1],),
                       prior=prior,
                       hp=hps,
                       enc_seed=best_enc_seed,
                       disc_seed=best_disc_seed,
                       enc_layers=hps['ae_number_of_layers'],
                       disc_layers=hps['disc_number_of_layers'],
                       with_density=False
                      )
testm.compile()

In [ ]:
#check layers
from asmsa.aae_model import _compute_number_of_neurons

neurons = _compute_number_of_neurons(layers=hps['ae_number_of_layers'], seed=best_enc_seed)
print(neurons)

In [ ]:
testm.summary()

In [ ]:
# specify earlystopping callback to avoid overfitting
monitored_metric = "AE loss min"

early_stop_cb = tf.keras.callbacks.EarlyStopping(
    monitor=monitored_metric,
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

In [ ]:
# train it (can be repeated several times to add more epochs)

testm.fit(X_train_batched,
          epochs=1000,
          verbose=2, # this flag is essential due to connection with EarlyStopping callback (epoch vs batch)
          callbacks=[
              early_stop_cb,
              visualizer.VisualizeCallback(testm,freq=100,inputs=X_train_np,figsize=(12,3))
          ])

In [ ]:
# Assuming 'monitored_metric' is defined and you have your history
since_epoch = 1

assert since_epoch > 0
history = np.array(testm.history.history[monitored_metric])
y = history[since_epoch-1:]
x = list(range(since_epoch, len(y) + since_epoch))

# Find the epochs with the minimum loss
result = np.array(list(map(lambda x: x + 1, np.where(history == history.min())[0])))  # Add +1 to convert index to epoch

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='AE Loss', color='blue', linewidth=2)

# Vertical lines for epochs with minimum loss
[plt.axvline(_x, linewidth=0.5, color='red', linestyle=':') for _x in result]

# Highlight the minimum loss value for annotation
min_loss_epoch = result[0]
min_loss_value = y[min_loss_epoch - since_epoch]

# Set limits for Y-axis
plt.ylim(bottom=min_loss_value)  # Start Y-axis at the minimum loss value

# Add gridlines
plt.grid(True)

# Add title and labels
plt.title(f'AE Loss During Training (Best Metric: {monitored_metric})')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# whatever test
''' 
batch_size = 256

val_result = testm.predict(X_test_batched)
mse = keras.losses.MeanSquaredError()
dataset_size = X_test_np.shape[0]
print(dataset_size)
mse_result=[]
for i in range(0, dataset_size, batch_size):
    if i+batch_size > dataset_size:
        batch_size = batch_size-(i+batch_size-dataset_size)
    batch_mse = mse(X_test_np[i:i+batch_size],val_result[i:i+batch_size]).numpy()
    mse_result.append(batch_mse)

mse_result'''

In [ ]:
# final visualization, pick a slice of the input data for demo purposes
#visualizer.Visualizer(figsize=(12,3)).make_visualization(testm.call_enc(X_train_np[15000:20000]).numpy())

# on test data
visualizer.Visualizer(figsize=(12,3)).make_visualization(testm.call_enc(X_test_np).numpy())

In [ ]:
# load testing trajectory for further visualizations and computations
tr = md.load('x_test.xtc',top=conf)
idx=tr[0].top.select("name CA")

# for trivial cases like Ala-Ala, where superposing on CAs fails
#idx=tr[0].top.select("element != H") 

tr.superpose(tr[0],atom_indices=idx)

# reshuffle the geometry to get frame last so that we can use vectorized calculations
geom = np.moveaxis(tr.xyz ,0,-1)
geom.shape

In [ ]:
# Rgyr and rmsd color coded in low dim (rough view)

lows = testm.call_enc(X_test_np).numpy()
rg = md.compute_rg(tr)
base = md.load(conf)
rmsd = md.rmsd(tr,base[0])
cmap = plt.get_cmap('rainbow')
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.scatter(lows[:,0],lows[:,1],marker='.',c=rg,cmap=cmap)
plt.colorbar(cmap=cmap)
plt.title("Rg")
plt.subplot(122)
plt.scatter(lows[:,0],lows[:,1],marker='.',c=rmsd,cmap=cmap)
plt.colorbar(cmap=cmap)
plt.title("RMSD")
plt.show()

## Save the model for Gromacs

*Another wave of magics ...*

There are multiple ways how atoms are numbered in PDB, GRO, etc. files. 

So far we worked with atoms numbered as in the `conf` PDB file, assuming `traj` XTC file was consistent with those.
If the topology was used, it might have had different numbering, as Gromacs likes. 

In the subsequent simulations, we assume the usual protocol starting with `pdb2gmx` to generate topology,
hence Gromacsish atom numbering will be followed afterwards.
Therefore we need `plumed.dat` to pick the atoms according to the PDB file order, and skip hydrogens added by Gromacs. 

Many things can go wrong, therefore we strongly encorage to check the results manually. For example, the first residuum (ASP) of tryptophan cage may look like the following in PDB file:

    ATOM      1  N   ASP     1      28.538  39.747  31.722  1.00  1.00           N
    ATOM      2  CA  ASP     1      28.463  39.427  33.168  1.00  1.00           C
    ATOM      3  C   ASP     1      29.059  37.987  33.422  1.00  1.00           C
    ATOM      4  O   ASP     1      30.226  37.748  33.735  1.00  1.00           O
    ATOM      5  CB  ASP     1      26.995  39.482  33.630  1.00  1.00           C
    ATOM      6  CG  ASP     1      26.889  39.307  35.101  1.00  1.00           C
    ATOM      7  OD1 ASP     1      27.749  39.962  35.773  1.00  1.00           O
    ATOM      8  OD2 ASP     1      26.012  38.510  35.611  1.00  1.00           O
    
Which turns Gromacs topology: 

     1         N3      1    ASP      N      1     0.0782      14.01   ; qtot 0.0782
     2          H      1    ASP     H1      2       0.22      1.008   ; qtot 0.2982
     3          H      1    ASP     H2      3       0.22      1.008   ; qtot 0.5182
     4          H      1    ASP     H3      4       0.22      1.008   ; qtot 0.7382
     5         CT      1    ASP     CA      5     0.0292      12.01   ; qtot 0.7674
     6         HP      1    ASP     HA      6     0.1141      1.008   ; qtot 0.8815
     7         CT      1    ASP     CB      7    -0.0235      12.01   ; qtot 0.858
     8         HC      1    ASP    HB1      8    -0.0169      1.008   ; qtot 0.8411
     9         HC      1    ASP    HB2      9    -0.0169      1.008   ; qtot 0.8242
    10          C      1    ASP     CG     10     0.8194      12.01   ; qtot 1.644
    11         O2      1    ASP    OD1     11    -0.8084         16   ; qtot 0.8352
    12         O2      1    ASP    OD2     12    -0.8084         16   ; qtot 0.0268
    13          C      1    ASP      C     13     0.5621      12.01   ; qtot 0.5889
    14          O      1    ASP      O     14    -0.5889         16   ; qtot 0
    
Besides adding hydrogens, the carboxyl group of the protein backbone (atoms 3,4 in PDB) is pushed down (to become 13,14 in the topology).

Consequently, the ATOMS setting in the generated `plumed.dat` must be:

    model: PYTORCH_MODEL_CV FILE=model.pt ATOMS=1,5,13,14,7,10,11,12, ...
    
i.e., the atoms are enumerated *in the order* of PDB file but *referring to numbers* of topology file. 

If there is any mismatch, the MD simulations are likely to fail, or at least to produce meaningless results.

It's also **critical** that `{conf}`, `{top}`, and `{gro}` correspond to one another, and that `{gro}` **includes hydrogens**.


In [ ]:
import tf2onnx
import onnx2torch
import tempfile

def _convert_to_onnx(model, destination_path):
#    model = keras.models.load_model(source_path)

    input_tensor = model.layers[0]._input_tensor
#    input_tensor = model.inputs[0]
    input_signature = tf.TensorSpec(
        name=input_tensor.name, shape=input_tensor.shape, dtype=input_tensor.dtype
    )
    output_name = model.layers[-1].name

    @tf.function(input_signature=[input_signature])
    def _wrapped_model(input_data):
        return {output_name: model(input_data)}

    tf2onnx.convert.from_function(
        _wrapped_model, input_signature=[input_signature], output_path=destination_path
    )

In [ ]:
model = testm

with tempfile.NamedTemporaryFile() as onnx:
#    tf2onnx.convert.from_keras(model.enc,output_path=onnx.name)
    _convert_to_onnx(model.enc,onnx.name)
    torch_encoder = onnx2torch.convert(onnx.name)

# load test geometry dataset
geom = np.stack(list(tf.data.Dataset.load('datasets/geoms/test')))

# XXX: we rely on determinism of the model creation, it must be the same as in prepare.ipynb
# better to store it there in onnx, and reload here

sparse_dists = asmsa.NBDistancesSparse(geom.shape[0], density=nb_density)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])
    
mol_model = mol.get_model()

train_mean = np.loadtxt('datasets/intcoords/mean.txt',dtype=np.float32)
train_scale = np.loadtxt('datasets/intcoords/scale.txt',dtype=np.float32)

def complete_model(x):
    return torch_encoder(
        ((mol_model(x) - torch.from_numpy(np.reshape(train_mean,(-1,1)))) / torch.from_numpy(np.reshape(train_scale,(-1,1)))).reshape(-1)
    )

# Save Torch model using TorchScript trace
example_input = torch.randn([geom.shape[0], geom.shape[1], 1])
traced_script_module = torch.jit.trace(complete_model, example_input)

model_file_name = "model2.pt"
traced_script_module.save(model_file_name)

In [ ]:
model=testm
with tempfile.NamedTemporaryFile() as onnx:
#    tf2onnx.convert.from_keras(model.enc,output_path=onnx.name)
    _convert_to_onnx(model.dec,onnx.name)
    torch_decoder = onnx2torch.convert(onnx.name)

example_input = torch.randn([1,2])
traced_script_module = torch.jit.trace(torch_decoder, example_input)

model_file_name = "decoder.pt"
traced_script_module.save(model_file_name)

In [ ]:
model_file_name = "model2.pt"
mod = torch.jit.load(model_file_name)

conf = "trpcage_npt400_nH.pdb"
traj = "trpcage_ds_nH.xtc"

tr = md.load(traj,top=conf)

stride = 1
lowdim = []
for i in range(0,tr.xyz.shape[0],stride):
    l = mod.forward(torch.from_numpy(tr.xyz[i].reshape(-1))).numpy()
    lowdim.append(l)
    
lowdim = np.array(lowdim)
lowdim.shape


base = md.load(conf)
rmsd = md.rmsd(tr,base[0])
cmap = plt.get_cmap('rainbow')



fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(lowdim[:,0],lowdim[:,1],marker='.', c=rmsd,cmap=cmap)#,c=range(0,tr.xyz.shape[0],stride))
#ax.scatter(lowdim[0,0],lowdim[0,1],marker='o', c='black',s=50)#,c=range(0,tr.xyz.shape[0],stride))
#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.set_ylabel('cv2', size="15")
ax.set_xlabel('cv1', size="15")
#plt.show()
plt.axis('on')

plt.savefig('model2.png')
np.savetxt('model2.txt', lowdim)

In [ ]:
# validate

example_geom = np.random.rand(geom.shape[0], geom.shape[1], 1)
#X = mol.intcoord(example_geom).T
X = ((mol.intcoord(example_geom) - np.reshape(train_mean,(-1,1))) / np.reshape(train_scale,(-1,1))).T
tf_low = np.array(model.enc(X))

torch_geom = torch.tensor(example_geom.reshape(-1), dtype=torch.float32, requires_grad=True)
torch_low = traced_script_module(torch_geom)

for out in torch_low:
    grad = torch.autograd.grad(out, torch_geom, retain_graph=True)

    
# should be very small, eg. less than 1e-5
np.max(np.abs(tf_low - torch_low.detach().numpy()))

In [ ]:
# Atom numbering magic with Gromacs, see above

grotr = md.load(gro)
nhs = grotr.topology.select('element != H')

with open(index) as f:
    f.readline()
    ndx = np.fromstring(" ".join(f),dtype=np.int32,sep=' ')-1

pdb2gmx = nhs[np.argsort(ndx)]+1

# maybe double check manually wrt. the files
# pdb2gmx

In [ ]:
ndx

In [ ]:
model_file_name = "model.pt"
mod = torch.jit.load(model_file_name)

conf = "villin_npt400_nh.pdb"
traj = "villin_ds_nh.xtc"

tr = md.load(traj,top=conf)

stride = 1
lowdim = []
for i in range(0,tr.xyz.shape[0],stride):
    l = mod.forward(torch.from_numpy(tr.xyz[i].reshape(-1))).numpy()
    lowdim.append(l)
    
lowdim = np.array(lowdim)
lowdim.shape


base = md.load(conf)
rmsd = md.rmsd(tr,base[0])
cmap = plt.get_cmap('rainbow')

fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(lowdim[:,0],lowdim[:,1],marker='.', c=rmsd,cmap=cmap)#,c=range(0,tr.xyz.shape[0],stride))
ax.scatter(lowdim[0,0],lowdim[0,1],marker='o', c='black',s=50)#,c=range(0,tr.xyz.shape[0],stride))
#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.set_ylabel('cv2', size="15")
ax.set_xlabel('cv1', size="15")
#plt.show()
plt.axis('on')

plt.savefig('model.png')

In [ ]:
np.savetxt('model.txt', lowdim)

#### Determine range of CVs for simulation

Plumed maintains a grid to approximate accumulated bias potential, which size must be known in advance.

Making it wider is safe, the simulation is less likely to escape and crash, but there is perfomance penalty.

Calculate the CVs on the testset, determine their range, and add some margins


In [ ]:
grid_margin = 1.  # that many times the actual computed size added on both sides

lows = model.call_enc(X_test_np).numpy()
lmin = np.min(lows,axis=0)
lmax = np.max(lows,axis=0)
llen = lmax-lmin
lmin -= llen * grid_margin
lmax += llen * grid_margin

In [ ]:
with open("plumed.dat","w") as p:
    p.write(f"""\
RESTART
WHOLEMOLECULES ENTITY0=1-{grotr.xyz.shape[1]}
model: PYTORCH_MODEL_CV FILE={model_file_name} ATOMS={','.join(map(str,pdb2gmx))}
metad: METAD ARG=model.node-0,model.node-1 PACE=500 HEIGHT=1 BIASFACTOR=15 SIGMA=0.2,0.2 GRID_MIN={lmin[0]},{lmin[1]} GRID_MAX={lmax[0]},{lmax[1]} FILE=HILLS
PRINT FILE=COLVAR ARG=model.node-0,model.node-1,metad.bias STRIDE=500
""")

## Decode

In [ ]:
hills = np.loadtxt('model2.txt')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

hb1 = ax1.hexbin(hills[:,0], hills[:,1], gridsize=50, cmap='seismic')
fig.colorbar(hb1, ax=ax1, label='Density')  
ax1.set_title('')
ax1.set_xlabel('cv1')
ax1.set_ylabel('cv2')

hb2 = ax2.hexbin(hills[:,0], hills[:,1], gridsize=50, cmap='tab10')
fig.colorbar(hb2, ax=ax2, label='Density')  
ax2.set_title('')
ax2.set_xlabel('cv1')
ax2.set_ylabel('cv2')

plt.tight_layout()
plt.show()

In [ ]:
model_file_name = "decoder.pt"
dec = torch.jit.load(model_file_name)

In [ ]:
sample1 = np.array([[3,3]])
sample1= torch.tensor(sample1, dtype=torch.float32)
d1 = dec(sample1)

In [ ]:
d1.shape

In [ ]:
np.save("decoded_structure.npy", d1)

In [ ]:
d1.shape

In [ ]:
print(d1[0,0,:150])

In [ ]:
geom.shape

In [ ]:
sparse_dists = asmsa.NBDistancesSparse(geom.shape[0], density=nb_density)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

In [ ]:
a = (0, len(mol.bonds))
mol.angles